[View in Colaboratory](https://colab.research.google.com/github/x110/MyFastaiNotebooks/blob/master/lang_model_arxiv.ipynb)

# Sentiment Classification


## Introduction

### Objective
The sentiment classification task consists of predicting the polarity (positive or negative) of a given text. 


###  Dataset
The [large moview view dataset](http://files.fast.ai/data/aclImdb.tgz) contains a collection of 50,000 reviews from IMDB. The datset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score $\leq 4$ out of 10, a positive review has a score $\geq 7$ out of 10. Nuteral reviews are not included in this dataset, 
The dataset we are using is the IMDb movie review dataset. "This is a dataset of binary sentiment classification. It contains a set of 25,000 higly polar movie reviews for training, and 25,000 for testing.  

### Solution Strategy
Before we try to classify sentiment, we will try to create a language model; that is, a model that can predict the next word in a sentence. Why? Because our model first needs to understand the structure of English, before we can expect it to recognize positive or negative sentiment. 

## Prepare virtual machine on google colab


1. Connect to google drive
2. Install dependencies

In [2]:
!ls

datalab


In [3]:
## Connect to google drive for storage 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
#Install dependencies
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install kaggle

    100% |████████████████████████████████| 90.0MB 433kB/s 
    100% |████████████████████████████████| 1.5MB 14.6MB/s 
    100% |████████████████████████████████| 112kB 15.0MB/s 
    100% |████████████████████████████████| 174kB 23.4MB/s 
    100% |████████████████████████████████| 3.4MB 11.7MB/s 


    100% |████████████████████████████████| 496.4MB 30kB/s 
    100% |████████████████████████████████| 51kB 11.6MB/s 
    100% |████████████████████████████████| 61kB 20.5MB/s 
    100% |████████████████████████████████| 51kB 12.5MB/s 
    100% |████████████████████████████████| 2.2MB 12.1MB/s 
    78% |█████████████████████████       | 10.7MB 35.4MB/s eta 0:00:01

    100% |████████████████████████████████| 13.7MB 3.5MB/s 
    100% |████████████████████████████████| 92kB 20.7MB/s 
    100% |████████████████████████████████| 112kB 20.7MB/s 
    100% |████████████████████████████████| 71kB 16.4MB/s 
    100% |████████████████████████████████| 81kB 22.5MB/s 
  Running setup.py bdist_wheel for fastai ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-52d4sbhb/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / -

 \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Running setup.py bdist_wheel for feather-format ... - \ done
  Stored in directory: /content/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for pandas-summary ... - done
  Stored in directory: /content/.cache/pip/wheels/86/a9/6b/36b727a9ec687dac08bd3d0501cddd7b1a223943513eb04a03
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built fastai bcolz feather-format pandas-summary torchtext
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 wh

    70% |██████████████████████▋         | 419.2MB 17.0MB/s eta 0:00:11

    100% |████████████████████████████████| 592.3MB 1.6MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: torch 0.3.1
    Uninstalling torch-0.3.1:
      Successfully uninstalled torch-0.3.1
    100% |████████████████████████████████| 2.0MB 14.6MB/s 
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/61/d6/e6/38fc0e2316f49b91aafdaf9a8d7756a9906b2ad7e501c1e165
Successfully built kaggle
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


In [5]:
#install More dependencies
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
!apt-get install p7zip-full

    100% |████████████████████████████████| 5.6MB 6.0MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.1MB 4.1MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
!mkdir .kaggle
!cp drive/kaggle.json /content/.kaggle/
!chmod 600 /content/.kaggle/kaggle.json

In [7]:
!pip install spacy

    100% |████████████████████████████████| 17.6MB 2.0MB/s 
    100% |████████████████████████████████| 92kB 13.1MB/s 
    100% |████████████████████████████████| 1.2MB 15.8MB/s 
    100% |████████████████████████████████| 51kB 18.1MB/s 
    100% |████████████████████████████████| 194kB 20.3MB/s 
    100% |████████████████████████████████| 153kB 21.3MB/s 
    100% |████████████████████████████████| 604kB 19.1MB/s 
    100% |████████████████████████████████| 389kB 22.6MB/s 
    100% |████████████████████████████████| 143kB 22.2MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / -

 \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ | / - \

 - \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


## Downloads

### Download the weights of the trained language model

In [0]:
#already in drive

### Download data

In [11]:
!wget http://files.fast.ai/data/aclImdb.tgz

--2018-06-27 13:36:19--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [text/plain]
Saving to: ‘aclImdb.tgz’

aclImdb.tgz         100%[===================>] 139.22M  35.5MB/s    in 4.5s    

2018-06-27 13:36:24 (31.0 MB/s) - ‘aclImdb.tgz’ saved [145982645/145982645]



In [0]:
!mkdir data

In [0]:
!tar -xzf aclImdb.tgz	

In [0]:
!mv aclImdb	data/

## Imports

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

## Language modeling
### Data

In [16]:
!ls

aclImdb.tgz  data  datalab  drive  Weights.zip


In [17]:
PATH = 'data/aclImdb/'
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

imdbEr.txt  imdb.vocab  README  test/  train/


In [18]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

In [19]:
review=!cat {TRN}{trn_files[6]}
review[0] 

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

In [20]:
!find {TRN} -name "*.txt" | xargs cat| wc -w

17486581


In [21]:
!find {VAL} -name "*.txt" | xargs cat | wc -w

5686719


In [22]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 26.9MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
spacy_tok=spacy.load('en')

In [24]:
' '.join(str(v) for v in spacy_tok(review[0]))

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop - socky fung - ku , but what I got instead was a comedy . So , it was n't quite was I was expecting , but I really liked it anyway ! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them ! ! I was laughing my ass off . I mean , the cops were just so bad ! And when I say bad , I mean The Shield Vic Macky bad . But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose ... man , oh man . What can you say about that hottie . She was great and put those other actresses to shame . She should work more often ! ! ! ! ! I also really liked the fight scene outside of the building . That was done really well . Lots of fighting and people getting their heads banged up . FUN ! Last , but not least Joe Estevez and William Smith were great as the ... well , I was n't sure what they were , but they see

In [0]:
TEXT=data.Field(lower=True,tokenize='spacy')#spacy_tok)#https://github.com/fastai/fastai/issues/213

In [26]:
type(list(spacy_tok(review[0]))[0])

spacy.tokens.token.Token

In [0]:
bs=64;bptt=70

In [0]:
FILES=dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)

In [29]:
%time md = LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=10)

CPU times: user 14min 15s, sys: 6.33 s, total: 14min 21s
Wall time: 14min 23s


In [30]:
md

In [34]:
!ls data/aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [0]:
!mkdir {PATH}models

pickle.dump(TEXT,open(f'{PATH}models/TEXT.pkl','wb'))

In [37]:
len(md.trn_dl),md.nt,len(md.trn_ds),len(md.trn_ds[0].text)

(4583, 37392, 1, 20540756)

This is the start of the mapping from integer IDs to unique tokens

In [38]:
#'itoi':'string to int'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [39]:
TEXT.vocab.stoi['the']

2

Note that in a languageModelData object there is only one item in each dataset: all words of the text joined together

In [40]:
md.trn_ds[0].text[:12]

['i',
 'know',
 'that',
 'this',
 'film',
 'is',
 'now',
 'at',
 'the',
 '10',
 'year',
 'mark']

torchtext will handle turning these words into integer IDs for us automatically

In [41]:
TEXT.numericalize([md.trn_ds[0].text[0:12]])

Variable containing:
   12
  138
   14
   13
   25
    9
  174
   40
    2
  328
  337
  997
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our language model object will create batches with 64 columns ,and varying sequence lengths of around 70 tokens.

Each batch also contains the exact sane data as labels,but one word later than the text since we are trying to predict the next words. the labels are flatten into a 1d array.

In [42]:
next(iter(md.trn_dl))

(Variable containing:
     12    269  10888  ...      11    540   1069
    138     10    585  ...      16     17     21
     14     28      4  ...     143  14779    699
         ...            ⋱           ...         
     77   5464      6  ...       4     15    656
     21   7642    919  ...    1745   6434     11
  10582     15      7  ...    3263     15      6
 [torch.cuda.LongTensor of size 65x64 (GPU 0)], Variable containing:
    138
     10
    585
   ⋮   
      9
    116
  27902
 [torch.cuda.LongTensor of size 4160 (GPU 0)])

## Train



In [0]:
em_sz=200#size of each embedding vector
nh=500 #number of hidden activation per layer
nl = 3 #number of layers

Researchers have found that large amounts of momentum dont work well with these kinds of RNN models, so we create a versio of adam optimizerwith less momentum than its default of .9

In [0]:
opt_fn = partial(optim.Adam,betas=(.7,.99))

fastai uses a variant of the state of the art AWD LSTM Language model developed by Stephen Merity. A key feature of that model is that it provides excellent regularization through dropout. 

The other parameters (alpha, beta, clip) should not generally need tuning.

In [0]:
learner = md.get_model(opt_fn,em_sz,nh,nl,
                      dropouti=.05, dropout = .05, wdrop=.1, dropoute=.02, dropouth=.05)
learner.reg_fn = partial(seq2seq_reg,alpha=2,beta=1)
learner.clip=.3

In [46]:
learner

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(37392, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(37392, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=37392)
    (dropout): LockedDropout(
    )
  )
)

In [77]:
#uncomment to refresh google drive
!google-drive-ocamlfuse -cc

Clearing cache...done


In [78]:
!ls drive

2018 Commandments.odt
alumni email.odt
Amazon instance setting.odt
Amazon.odt
Attendance.ods
August 17, 2017.odt
caffe configuration.odt
codeAWS.zip
Colab Notebooks
consistent.ods
Convex optimization Exercises.odt
convex optimization.odt
Copy of Copy of fifthJune182018.ods
Copy of fifth.ods
Copy of Mariam Hours Sheet.ods
Copy of Mariam Hours Sheet.ods (3e583114)
Copy of Mariam Hours Sheet.ods (50be9b93)
Copy of Mariam Hours Sheet.ods (6c49228c)
Copy of SOP5.odt
Dataset.pkl
Do Before new semester.odt
EuroSimE_Abstract_Template.doc
EuroSimE_Abstract_Template.doc.odt
Feedback to Reviewers_Manuscript Rb7694_v1.odt
fifth.ods
Goals:Nov30-Dec6.odt
goals.odt
Home_Depot_Product_Search_Relevance2.zip
imdb2x_cyc_4.h5
imdb_adam3_c1_cl10_cyc_0.h5
install cuda+mxnet.odt
Install opencv2.odt
Intelligent_commerce.pdf
JanTrain12_0.rec
JanTrain12_1 (344f4ae1).rec
JanTrain12_1 (5d119f5e).rec
JanTrain12_1.rec
JanValid12_0.rec
Job Search.ods
Journal2.zip
Kaggle - April.odt
kaggle.json
Kaggle.odt
mariam cv.d

In [89]:
!cp drive/imdb_adam3_c1_cl10_cyc_0.h5 data/aclImdb/models/
learner.load('imdb_adam3_c1_cl10_cyc_0')

RuntimeError: ignored

In [90]:
!ls drive/*.h5

drive/imdb2x_cyc_4.h5  drive/imdb_adam3_c1_cl10_cyc_0.h5  drive/resnet_75.h5
